Temat 3: Text Mining
Zbiór Danych: sentyment_movies.csv zawiera recenzję filmów z portalu branżowego, wraz ze zmienną zawierającą informację, czy recenzja jest pozytywna, czy negatywna.
•	Wskaż, które słowa są najbardziej charakterystyczne dla recenzji pozytywnych, a które dla negatywnych.
•	Pamiętaj o filtrowaniu słów stanowiących szum, oraz o wybieraniu tych kategorii słów, które mogą być adekwatne do zadania.
•	Za pomocą znanych Ci metod uczenia maszynowego sprawdź, czy istnieją jakieś zgrupowania współwystępujących ze sobą słów.
Wyniki mogą być przedstawione w formie notatnika w Jupyter, albo w formie raporty z użyciem python Dash


https://www.kaggle.com/oumaimahourrane/sentiment-analysis-ml-models-comparison
https://www.kaggle.com/oumaimahourrane/imdb-reviews/kernels
https://www.kaggle.com/sergiadi/iet-x-mlda-workshop